# Imports and inits

In [32]:
import numpy as np
import sympy as sp
from sympy import cos, sin

In [40]:
t = sp.Symbol('t')
qs = [sp.Function(f'q_{i}')(t) for i in range(6)] # Joints effect
ls = sp.symbols(r"l_:6") # Link lengths
hs = sp.symbols(r"h_:3") # Link heights
ws = sp.symbols(r"w_:3") # Link widths
rs = sp.symbols(r"r_:6") # Link radii

In [41]:
qs

[q_0(t), q_1(t), q_2(t), q_3(t), q_4(t), q_5(t)]

In [42]:
# Source: symbolic_fk.ipynb demo code

def rz(theta):
    rot = sp.zeros(4,4)
    rot[3,3] = 1
    rot[0,:] = [[cos(theta), -sin(theta),0,0]]
    rot[1,:] = [[sin(theta), cos(theta),0,0]]
    rot[2,2] = 1

    return rot

def rx(theta): 
    rot = sp.zeros(4,4)
    rot[3,3] = 1
    rot[0,0] = 1
    rot[1,:] = [[0, cos(theta), -sin(theta),0]]
    rot[2,:] = [[0, sin(theta), cos(theta),0]]

    return rot


def ry(theta): 
    rot = sp.zeros(4,4)
    rot[3,3] = 1
    rot[1,1] = 1
    rot[0,:] = [[ cos(theta), 0, sin(theta),0]]
    rot[2,:] = [[-sin(theta),0, cos(theta),0]]

    return rot


def trans(vector):
    mat = sp.eye(4)
    mat[0:3,3] = vector
    return mat

def tx(dx):
    return trans([dx, 0, 0])

def ty(dy):
    return trans([0, dy, 0])

def tz(dz):
    return trans([0, 0, dz])

# Calculating transformation

In [63]:
dh_table = [
    [0, 0, 0, 0],
    [sp.pi / 2, sp.pi / 2, 0, qs[1] + ls[1]],
    [0, -sp.pi / 2, 0, qs[2] + ls[2]],
    [-sp.pi / 2 + qs[3], sp.pi / 2, 0, ls[3]],
    [sp.pi / 2 + qs[4], sp.pi / 2, 0, 0],
    [qs[5], 0, 0, ls[4]]
]

In [67]:
def dh_to_matrix(dh_row):
    theta, alpha, a, d = dh_row
    return rz(theta) * tx(a) * tz(d) * rx(alpha)

In [107]:
print('All consequent transformations:')
T = sp.eye(4)
for i in range(len(dh_table)):
    print(f'{i} -> {i + 1}')
    res = dh_to_matrix(dh_table[i])
    T = T * res
    display(res)

print('Total transformation:')
display(T)

All consequent transformations:
0 -> 1


Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]])

1 -> 2


Matrix([
[0, 0, 1,            0],
[1, 0, 0,            0],
[0, 1, 0, l_1 + q_1(t)],
[0, 0, 0,            1]])

2 -> 3


...

Matrix([
[-sin(q_4(t)), 0, cos(q_4(t)), 0],
[ cos(q_4(t)), 0, sin(q_4(t)), 0],
[           0, 1,           0, 0],
[           0, 0,           0, 1]])

5 -> 6


Matrix([
[cos(q_5(t)), -sin(q_5(t)), 0,   0],
[sin(q_5(t)),  cos(q_5(t)), 0,   0],
[          0,            0, 1, l_4],
[          0,            0, 0,   1]])

Total transformation:


Matrix([
[ sin(q_3(t))*sin(q_5(t)) - sin(q_4(t))*cos(q_3(t))*cos(q_5(t)), sin(q_3(t))*cos(q_5(t)) + sin(q_4(t))*sin(q_5(t))*cos(q_3(t)), cos(q_3(t))*cos(q_4(t)), l_2 + l_4*cos(q_3(t))*cos(q_4(t)) + q_2(t)],
[-sin(q_3(t))*sin(q_4(t))*cos(q_5(t)) - sin(q_5(t))*cos(q_3(t)), sin(q_3(t))*sin(q_4(t))*sin(q_5(t)) - cos(q_3(t))*cos(q_5(t)), sin(q_3(t))*cos(q_4(t)),                l_4*sin(q_3(t))*cos(q_4(t))],
[                                       cos(q_4(t))*cos(q_5(t)),                                      -sin(q_5(t))*cos(q_4(t)),             sin(q_4(t)),       l_1 + l_3 + l_4*sin(q_4(t)) + q_1(t)],
[                                                             0,                                                             0,                       0,                                          1]])

In [81]:
print('Translation:')
display(T[0:3,3])
print('Rotation:')
display(T[0:3,0:3])

Translation:


Matrix([
[l_2 + l_4*cos(q_3(t))*cos(q_4(t)) + q_2(t)],
[               l_4*sin(q_3(t))*cos(q_4(t))],
[      l_1 + l_3 + l_4*sin(q_4(t)) + q_1(t)]])

Rotation:


Matrix([
[ sin(q_3(t))*sin(q_5(t)) - sin(q_4(t))*cos(q_3(t))*cos(q_5(t)), sin(q_3(t))*cos(q_5(t)) + sin(q_4(t))*sin(q_5(t))*cos(q_3(t)), cos(q_3(t))*cos(q_4(t))],
[-sin(q_3(t))*sin(q_4(t))*cos(q_5(t)) - sin(q_5(t))*cos(q_3(t)), sin(q_3(t))*sin(q_4(t))*sin(q_5(t)) - cos(q_3(t))*cos(q_5(t)), sin(q_3(t))*cos(q_4(t))],
[                                       cos(q_4(t))*cos(q_5(t)),                                      -sin(q_5(t))*cos(q_4(t)),             sin(q_4(t))]])

In [106]:
def forward(i, j):
    T = sp.eye(4)
    for k in range(i, j):
        T = T * dh_to_matrix(dh_table[k])
    return T

# Calculating Jacobian

In [135]:
w1 = sp.Matrix([0, 0, 0]) # local axis of joint 1
w2 = sp.Matrix([0, 0, 0]) # local axis of joint 2
w3 = sp.Matrix([0, 0, 1]) # local axis of joint 3
w4 = sp.Matrix([0, 1, 0]) # local axis of joint 4
w5 = sp.Matrix([1, 0, 0]) # local axis of joint 5

R1 = forward(0, 4)[0:3,0:3]
R2 = forward(0, 5)[0:3,0:3]
R3 = forward(0, 6)[0:3,0:3]

w3 = R1 @ w3
w4 = R2 @ w4
w5 = R3 @ w5

In [142]:
display(w3)

Matrix([
[ sin(q_3(t))],
[-cos(q_3(t))],
[           0]])

In [178]:
J_w = []
for i in range(3):
    J_w.append([w1[i], w2[i], w3[i], w4[i], w5[i]])
J_w = sp.Matrix(J_w)

J_v = []

P = forward(0, 6)[0:3,3]

for j in range(3):
    J_v.append([])
    for k in range(5):
        J_v[j].append(sp.Derivative(P[j], qs[k + 1]).doit())
J_v = sp.Matrix(J_v)

In [179]:

print('Angular velocity Jacobian:')
display(J_w)
print('Linear Velocity Jacobian:')
display(J_v)
print('Jacobian:')
J = J_v.col_join(J_w)
display(J)

Angular velocity Jacobian:


Matrix([
[0, 0,  sin(q_3(t)),  sin(q_3(t)),  sin(q_3(t))*sin(q_5(t)) - sin(q_4(t))*cos(q_3(t))*cos(q_5(t))],
[0, 0, -cos(q_3(t)), -cos(q_3(t)), -sin(q_3(t))*sin(q_4(t))*cos(q_5(t)) - sin(q_5(t))*cos(q_3(t))],
[0, 0,            0,            0,                                        cos(q_4(t))*cos(q_5(t))]])

Linear Velocity Jacobian:


Matrix([
[0, 1, -l_4*sin(q_3(t))*cos(q_4(t)), -l_4*sin(q_4(t))*cos(q_3(t)), 0],
[0, 0,  l_4*cos(q_3(t))*cos(q_4(t)), -l_4*sin(q_3(t))*sin(q_4(t)), 0],
[1, 0,                            0,              l_4*cos(q_4(t)), 0]])

Jacobian:


Matrix([
[0, 1, -l_4*sin(q_3(t))*cos(q_4(t)), -l_4*sin(q_4(t))*cos(q_3(t)),                                                              0],
[0, 0,  l_4*cos(q_3(t))*cos(q_4(t)), -l_4*sin(q_3(t))*sin(q_4(t)),                                                              0],
[1, 0,                            0,              l_4*cos(q_4(t)),                                                              0],
[0, 0,                  sin(q_3(t)),                  sin(q_3(t)),  sin(q_3(t))*sin(q_5(t)) - sin(q_4(t))*cos(q_3(t))*cos(q_5(t))],
[0, 0,                 -cos(q_3(t)),                 -cos(q_3(t)), -sin(q_3(t))*sin(q_4(t))*cos(q_5(t)) - sin(q_5(t))*cos(q_3(t))],
[0, 0,                            0,                            0,                                        cos(q_4(t))*cos(q_5(t))]])

# Nullspaces

In [174]:
J.nullspace()

[]

In [169]:
for vec in J[0:3, :].nullspace():
    print('Nullspace vector:')
    display(sp.simplify(vec))

Nullspace vector:


Matrix([
[           -l_4*cos(q_4(t))],
[l_4*sin(q_4(t))/cos(q_3(t))],
[    tan(q_3(t))*tan(q_4(t))],
[                          1],
[                          0]])

Nullspace vector:


Matrix([
[0],
[0],
[0],
[0],
[1]])

In [182]:
sp.simplify(J.T.nullspace()[0])

Matrix([
[                      0],
[                      0],
[                      0],
[cos(q_3(t))/tan(q_4(t))],
[sin(q_3(t))/tan(q_4(t))],
[                      1]])

# Simulation
Given that we know, that derivative of generalized actuations is basically the nullspace of the Jacobian, we can use the following formula to calculate corresponding velocities:
(to simplify calculations I have assumed all the links to be equal in length to 1, all integration constants to be zero and all generalized actuations to be equal to 0 where possible)

In [213]:
q1 = -1 * sp.cos(t)
q2 = 1 * sp.sin(t)
q3 = 0
q4 = t
q5 = 0

In [263]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation, PillowWriter

plt.rcParams.update({
    "savefig.facecolor": (1.0, 1.0, 1.0, 1)
})

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.set_xlim(-1, 2.2)
ax.set_ylim(-1, 2.2)
ax.set_zlim(-1, 2.2)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
xs = [0.0, 0.0, 1.0, 1.0, 2.0]
ys = [0.0, 0.0, 0.0, 0.0, 0.0]
zs = [0.0, 1.0, 1.0, 2.0, 2.0]

mech_joints = []
mech_links = []
mech_labels = []
prev_frame = 0

def update_model(frame):
    global prev_frame
    
    dt = frame - prev_frame
    for i in range(len(zs)):
        zs[i] += dt * float(q1.subs(t, frame).evalf())
        if i > 0:
            xs[i] += dt * float(q2.subs(t, frame).evalf())
        
    
    xs[4] += np.cos(frame) - np.cos(prev_frame)
    zs[4] += np.sin(frame) - np.sin(prev_frame)
    
    prev_frame = frame

    while not len(mech_links) == 0:
        mech_links[-1][0].remove()
        mech_links.pop()

    while not len(mech_joints) == 0:
        mech_joints[-1].remove()
        mech_joints.pop()

    while not len(mech_labels) == 0:
        mech_labels[-1].remove()
        mech_labels.pop()

    mech_labels.append(ax.text(0, 0, -0.05, '$trans_1$', fontsize=10))
    mech_labels.append(ax.text(0, ys[2], zs[2] + 0.05, '$trans_2$', fontsize=10))
    mech_labels.append(ax.text(xs[2], ys[2], zs[2] - 0.05, '$rev_1$', fontsize=10))
    mech_labels.append(ax.text(xs[3] + 0.05, ys[3], zs[3], '$rev_2$', fontsize=10))
    mech_labels.append(ax.text(xs[4], ys[4], zs[4] + 0.05, '$rev_3$', fontsize=10))
    mech_joints.append(ax.scatter(xs[2:], ys[2:], zs[2:], color='blue'))
    mech_joints.append(ax.scatter([0], ys[2], zs[2], color='blue'))
    mech_joints.append(ax.scatter([0], [0], [0], color='blue'))
    mech_links.append(ax.plot([0, 0], [0, ys[2]], [0, zs[2]], color='red'))
    mech_links.append(ax.plot(xs[2:5], ys[2:5], zs[2:5], color='red'))
    mech_links.append(ax.plot([0, xs[2]], [ys[2], ys[2]], [zs[2], zs[2]], color='red'))

    return []

anim = FuncAnimation(fig, update_model, frames=np.linspace(0, 10, 300), blit=True)

anim.save('task2.gif', dpi=300, writer=PillowWriter(fps=60))
plt.close('all')